<a href="https://colab.research.google.com/github/yugonojima/Resnet-WideResnet-model-/blob/main/Resnet_%26_WideResnet_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resnet

残差ブロック

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as kl

In [ ]:
class Res_Block(tf.keras.Model):
    def __init__ (self, in_channels, out_channels):
        super().__init__()

        bneck_channels = out_channels // 4

        self.bn1 = kl.BatchNormalization()
        self.av1 = kl.Activation(tf.nn.relu)
        self.conv1 = kl.Conv2D(bneck_channels, kernes_size = 1, strides = 1, padding = 'valid', use_bias = False)
        
        self.bn2 = kl.BatchNormalization()
        self.av2 = kl.Activation(tf.nn.relu)
        self.conv2 = kl.Conv2D(bneck_channels, kernes_size = 3, strides = 1, padding = 'same', use_bias = False)
        
        self.bn3 = kl.BatchNormalization()
        self.av3 = kl.Activation(tf.nn.relu)
        self.conv3 = kl.Conv2D(out_channels, kernes_size = 1, strides = 1, padding = 'valid', use_bias = False)

        self.shortcut = self._scblock(in_channels, out_channels)
        self.add = kl.Add()

    def _scblock(self, in_channels, out_channels):
        if in_channels != out_channels:
            self.bn_sc1 = kl.BatchNormalization()
            self.conv_sc1 = kl.Conv2D(out_channels, kernel_size= 1, strides = 1, padding= 'same', use_bias=False)

            return self.conv_sc1
        else :
            return lambda x : x #xをxのまま返す関数

    def call(self, x):
         out1 = self.conv1(self.av1(self.bn1(x)))
         out2 = self.conv2(self.av2(self.bn2(out1)))        
         out3 = self.conv3(self.av3(self.bn3(out2)))
         shortcut = self.shortcut(x)    
         out4 = self.add([out3, shortcut])

         return out4



Wide Resnet

In [ ]:
import numpy as np 
import os 
import os.path as path
import shutil
import pickle

from tensorflow.keras.utils import to_categorical
from keras.utils.data_utils import get_file
from operator import attrgetter


In [ ]:
from six import b
def download_data():
    shutil.copytree(get_file('cifar-10-batches-py', origin='http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz', untar=True), './data')

def load_data(data_path='./data'):
    def load_batch(path):
        with open(path, 'rb') as f: #withを利用することで、ファイルを閉じる処理を行わなくてもファイルを閉じる終了処理を実行してくれる
                                   #'rb'はバイナリファイルの読み込み
            batch = pickle.load(f, encoding='bytes')#load関数: pickle化(バイナリファイルから読み込んだデータはpickle化されている)されたデータを読み込んで非pickle化する

        return (np.array(batch[b'data']).reshape(batch[b'data'].shape[0], 3, 32, 32).transpose(0,2,3,1) /255, to_categorical(np.array(batch[b'labels'])))[:1000]

    def load_batches(paths):
            return tuple(map(np.concatenate, zip(*map(load_batch, paths))))

    if not path.exists('./data'):
            download_data()

    return (load_batches(sorted(map(attrgetter('path'), filter(lambda directory_entry: directory_entry.name.startswith('data_batch_'), os.scandir(data_path))))), 
            load_batch('{0}/test_batch'.format(data_path)))

In [ ]:
!pip install funcy

In [ ]:
import numpy as np
import pickle

from funcy                     import concat, identity, juxt, partial, rcompose, repeat, repeatedly, take
from keras.callbacks           import LearningRateScheduler
from keras.layers              import Activation, Add, BatchNormalization, Conv2D, Dense, GlobalAveragePooling2D, Input
from keras.models              import Model, save_model
from tensorflow.keras.optimizers          import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers        import l2
from tensorflow.keras.utils               import plot_model
from operator                  import getitem

#rcompose: 合成関数を簡単に記述することができる関数,前から順に計算される

def computational_graph(class_size):

    def ljuxt(*fs): #juxt: 同じ引数を持つ二つの関数からの返り値を受け取ることができる関数
                   #juxtは返り値がジェネレーターとなるため、ジェネレーターではなく、listで返り値を返せるように定義
        return rcompose(juxt(*fs), list)

    def batch_normalization():
        return BatchNormalization()

    def relu():
        return Activation('relu')

    def conv(filter_size, kernel_size, stride_size=1):
        return Conv2D(filter_size, kernel_size, strides=stride_size, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), use_bias=False)  # ReLUするならウェイトをHe初期化するのが基本らしい。あと、Kerasにはweight decayがなかったのでkernel_regularizerで代替したのたけど、これで正しい？

    def add():
        return Add()

    def global_average_pooling():
        return GlobalAveragePooling2D()

    def dense(unit_size, activation):
        return Dense(unit_size, activation=activation, kernel_regularizer=l2(0.0005))  

    def first_residual_unit(filter_size, stride_size):
        return rcompose(batch_normalization(),
                        relu(),
                        ljuxt(rcompose(conv(filter_size, 3, stride_size),
                                       batch_normalization(),
                                       relu(),
                                       conv(filter_size, 3, 1)),
                              rcompose(conv(filter_size, 1, stride_size))),
                        add())

    def residual_unit(filter_size):
        return rcompose(ljuxt(rcompose(batch_normalization(),
                                       relu(),
                                       conv(filter_size, 3),
                                       batch_normalization(),
                                       relu(),
                                       conv(filter_size, 3)),
                              identity),
                        add())

    def residual_block(filter_size, stride_size, unit_size):
        return rcompose(first_residual_unit(filter_size, stride_size),
                        rcompose(*repeatedly(partial(residual_unit, filter_size), unit_size - 1)))

    k = 10 
    n =  4 

    return rcompose(conv(16, 3),
                    residual_block(16 * k, 1, n),
                    residual_block(32 * k, 2, n),
                    residual_block(64 * k, 2, n),
                    batch_normalization(),
                    relu(),
                    global_average_pooling(),
                    dense(class_size, 'softmax'))


def main():
    (x_train, y_train), (x_validation, y_validation) = load_data()

    model = Model(*juxt(identity, computational_graph(y_train.shape[1]))(Input(shape=x_train.shape[1:])))
    model.compile(loss='categorical_crossentropy', optimizer=SGD(momentum=0.9), metrics=['accuracy']) 

    model.summary()

    train_data      = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, width_shift_range=0.125, height_shift_range=0.125, horizontal_flip=True)
    validation_data = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)

    for data in (train_data, validation_data):
        data.fit(x_train) 

    batch_size = 128
    epoch_size = 1

    results = model.fit_generator(train_data.flow(x_train, y_train, batch_size=batch_size),
                                  steps_per_epoch=x_train.shape[0] // batch_size,
                                  epochs=epoch_size,
                                  callbacks=[LearningRateScheduler(partial(getitem, tuple(take(epoch_size, concat(repeat(0.1, 60), repeat(0.02, 60), repeat(0.004, 40), repeat(0.0008))))))],
                                  validation_data=validation_data.flow(x_validation, y_validation, batch_size=batch_size),
                                  validation_steps=x_validation.shape[0] // batch_size)

    # with open('./results/history.pickle', 'wb') as f:
    #     pickle.dump(results.history, f)

    # save_model(model, './results/model.h5')


# if __name__ == '__main__':
    main()